In [1]:
import pandas as pd
import numpy as np
import keras
from keras import layers
from sklearn.metrics import (confusion_matrix, precision_score, auc,
                             roc_curve, recall_score, classification_report, f1_score)
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import random
import scipy.stats as stats
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import csv
from sklearn.cluster import KMeans
from sklearn_som.som import SOM
from sklearn import svm
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn import tree
import glob
import sys
np.set_printoptions(threshold=sys.maxsize)

# Dados NSL-KDD

In [2]:
data = pd.read_csv(r"C:\Users\luang\Downloads\BRACIS\UNSW_2018_IoT_Botnet_Full5p.csv")
x = data.drop (['category', 'subcategory', 'attack'], axis = 1)
y = data ['attack']

In [4]:
x['flgs'].unique()

array([0, 1, 5, 4, 2, 3, 6, 7, 8], dtype=int64)

In [3]:
data = x.to_numpy()
data_class = y.to_numpy()

# Abordagem Autoencoder-KNN

In [4]:
epochs = 50
batch_size = 256
optimizer = Adam(learning_rate=0.0001)
activation = "relu"
neighbors = 11

num_folds = 10

kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)

fold_no = 1
count = 0

#Métricas ErroRecons
matrix_ErroRecons = np.zeros((num_folds,2,2))
accuracy_ErroRecons = np.zeros(num_folds)
recall_ErroRecons = np.zeros(num_folds)
precision_ErroRecons = np.zeros(num_folds)
f1_ErroRecons = np.zeros(num_folds)
Z = np.arange(0.000,4.095,0.005)
bestZ = np.zeros(num_folds)
bestTeta = np.zeros(num_folds)
bestmean = np.zeros(num_folds)
bestdesv = np.zeros(num_folds)
bestErrorNumber = np.zeros(num_folds)
ErrorNumberFolds = np.zeros((num_folds,1,Z.shape[0]-1))

#Métricas KNN
matrix_knn = np.zeros((num_folds,2,2))
accuracy_knn = np.zeros(num_folds)
recall_knn = np.zeros(num_folds)
precision_knn = np.zeros(num_folds)
f1_knn = np.zeros(num_folds)

#Métricas Decision Tree
matrix_dt = np.zeros((num_folds,2,2))
accuracy_dt = np.zeros(num_folds)
recall_dt = np.zeros(num_folds)
precision_dt = np.zeros(num_folds)
f1_dt = np.zeros(num_folds)

#Métricas SVM
matrix_svm = np.zeros((num_folds,2,2))
accuracy_svm = np.zeros(num_folds)
recall_svm = np.zeros(num_folds)
precision_svm = np.zeros(num_folds)
f1_svm = np.zeros(num_folds)

for train, test in kfold.split(data, data_class):
    
    #####################################################
    ####Pré-Processamento
    
    #Separação dos Folds de Treinamento e Teste
    dataTrain = data[train]
    dataTrain_class = data_class[train]
    modelTest = data[test]
    modelTest_class = data_class[test]
    
    #Conta Quantidade de dados
    Train = dataTrain.shape[1]
    noTrain = np.count_nonzero(dataTrain_class == 0)
    anTrain = np.count_nonzero(dataTrain_class == 1)

    #Separa dados normais e anômalos
    normalTrain = dataTrain[np.where(dataTrain_class == 0)]
    anomalyTrain = dataTrain[np.where(dataTrain_class == 1)]
    
    #Divide dados para treinamento
    porcen = 1
    j = int(noTrain * porcen)
    
    #Índices escolhidos aleatoriamente
    numbers_knn_normal = np.array(random.sample(range(0, noTrain),j))
    numbers_knn_anomaly = np.array(random.sample(range(0,anTrain),j))
    numbers_autoencoder_anomaly = np.array(list(set(np.arange(0, anTrain)) - set(numbers_knn_anomaly)))
    
    #Dados KNN
    knnTrain_normal = normalTrain[numbers_knn_normal]
    knnTrain_anomaly = anomalyTrain[numbers_knn_anomaly]
    knnTrain = np.concatenate((knnTrain_normal, knnTrain_anomaly), axis=0)
    knnTrain_class = np.concatenate((np.zeros(j), np.ones(j)))
    
    #Dados Autoencoder
    autoencoderTrain = anomalyTrain[numbers_autoencoder_anomaly]
    autoencoderTrain_class = np.ones(autoencoderTrain.shape[0])
    
    #Normalização
    scaler = MinMaxScaler()
    scaler.fit(knnTrain)
    autoencoderTrain = scaler.transform(autoencoderTrain)
    knnTrain = scaler.transform(knnTrain)
    modelTest = scaler.transform(modelTest)
    
    #####################################################
    ####Autoencoder
    #Modela as camadas do Autoencoder
    var = autoencoderTrain.shape[1]
    input_vector = keras.Input(shape=(var,))
    en1 = layers.Dense(int(var/2), activation=activation)(input_vector)
    en2 = layers.Dense(int(var/4), activation=activation)(en1)
    de1 = layers.Dense(int(var/2), activation=activation)(en2)
    de2 = layers.Dense(var, activation=activation)(de1)

    #Gera o modelo
    autoencoder = keras.Model(input_vector, de2)

    #Compila o modelo
    autoencoder.compile(optimizer=optimizer, loss="mean_squared_error")

    #Treina o modelo
    history = autoencoder.fit(autoencoderTrain, autoencoderTrain, epochs = epochs, batch_size = batch_size, shuffle = True)
    
    #####################################################
    ####Erro de Reconstrução
    #Teste
    pred_ErroRecons = autoencoder.predict(modelTest)
    
    #Extração do erro
    mse_ErroRecons = np.mean(np.power(modelTest - pred_ErroRecons, 2), axis=1)
    error_df_ErroRecons = pd.DataFrame({'reconstruction_error': mse_ErroRecons,
                                        'true_class': modelTest_class})

    media = np.mean(error_df_ErroRecons['reconstruction_error'])
    desv = np.std(error_df_ErroRecons['reconstruction_error'])

    #Determinação do limiar ótimo

    ErrosLimiar = np.zeros(Z.shape[0]-1)
    melhorTeta = 0
    melhorZ = 0
    prediMatrix = np.zeros((Z.shape[0]-1, 1, modelTest.shape[0]))
    
    for i in range(0, Z.shape[0]-1):
        #Obtenção do Limiar
        teta = media + Z[i]*desv
    
        #Predição
        predi_ErroRecons = np.ones(modelTest.shape[0])
        error_ErroRecons = error_df_ErroRecons.to_numpy()
        predi_ErroRecons[np.where(error_ErroRecons[:,0] <= teta)] = 0
        prediMatrix[i] = predi_ErroRecons 
        
        #Contagem da quantidade de erros obtidos para cada Z
        numberErros = 0
        for j in range(0, predi_ErroRecons.shape[0]):
            if error_ErroRecons[j,1] != predi_ErroRecons[j]:
                numberErros = numberErros + 1
        ErrosLimiar[i] = numberErros
        
    predi_ErroRecons = prediMatrix[np.argpartition(ErrosLimiar,1)[0]]
            
    #Métricas
    accuracy_ErroRecons[count] = accuracy_score(error_ErroRecons[:,1], predi_ErroRecons[:].reshape(-1,1))
    recall_ErroRecons[count] = recall_score(error_ErroRecons[:,1], predi_ErroRecons[:].reshape(-1,1))
    precision_ErroRecons[count] = precision_score(error_ErroRecons[:,1], predi_ErroRecons[:].reshape(-1,1))
    f1_ErroRecons[count] = f1_score(error_ErroRecons[:,1], predi_ErroRecons[:].reshape(-1,1))
    matrix_ErroRecons[count] = confusion_matrix(error_ErroRecons[:,1], predi_ErroRecons[:].reshape(-1,1))
    bestZ[count] = Z[np.argpartition(ErrosLimiar,1)[0]]
    bestTeta[count] = media + Z[np.argpartition(ErrosLimiar,1)[0]]*desv
    bestmean[count] = media
    bestdesv[count] = desv
    bestErrorNumber[count] = ErrosLimiar[np.argpartition(ErrosLimiar,1)[0]]
    ErrorNumberFolds[count] = ErrosLimiar
    
    #####################################################
    ####KNN
    ###Treinamento KNN
    pred_knn = autoencoder.predict(knnTrain)
    mse_knn = np.mean(np.power(knnTrain - pred_knn, 2), axis=1)
    error_df_knn = pd.DataFrame({'reconstruction_error': mse_knn,
                                 'true_class': knnTrain_class})
    error_knn = error_df_knn.to_numpy()
    neigh = KNeighborsClassifier(n_neighbors=neighbors)
    neigh.fit(error_knn[:,0].reshape(-1,1), error_knn[:,1])
    
    #Teste do Modelo
    pred_knn = autoencoder.predict(modelTest)
    mse_knn = np.mean(np.power(modelTest - pred_knn, 2), axis=1)
    error_df_knn = pd.DataFrame({'reconstruction_error': mse_knn,
                                'true_class': modelTest_class})
    error_knn = error_df_knn.to_numpy()
    predict_knn = neigh.predict(error_knn[:,0].reshape(-1,1))
    
    #Métricas
    accuracy_knn[count] = accuracy_score(error_knn[:,1], predict_knn[:])
    recall_knn[count] = recall_score(error_knn[:,1], predict_knn[:])
    precision_knn[count] = precision_score(error_knn[:,1], predict_knn[:])
    f1_knn[count] = f1_score(error_knn[:,1], predict_knn[:])
    matrix_knn[count] = confusion_matrix(error_knn[:,1], predict_knn[:])
    
      
    #####################################################
    ####Decision Tree
    ###Treinamento Decision Tree 
    pred_dt = autoencoder.predict(knnTrain)
    mse_dt = np.mean(np.power(knnTrain - pred_dt, 2), axis=1)
    error_df_dt = pd.DataFrame({'reconstruction_error': mse_dt,
                                     'true_class': knnTrain_class})
    error_dt = error_df_dt.to_numpy()
    dt = tree.DecisionTreeClassifier()
    dt.fit(error_dt[:,0].reshape(-1,1), error_dt[:,1])

    #Teste
    pred_dt = autoencoder.predict(modelTest)
    mse_dt = np.mean(np.power(modelTest - pred_dt, 2), axis=1)
    error_df_dt = pd.DataFrame({'reconstruction_error': mse_dt,
                                     'true_class': modelTest_class})
    error_dt = error_df_dt.to_numpy()
    predict_dt = dt.predict(error_dt[:,0].reshape(-1,1))

    #Métricas
    accuracy_dt[count] = accuracy_score(error_dt[:,1], predict_dt[:])
    recall_dt[count] = recall_score(error_dt[:,1], predict_dt[:])
    precision_dt[count] = precision_score(error_dt[:,1], predict_dt[:])
    f1_dt[count] = f1_score(error_dt[:,1], predict_dt[:])
    matrix_dt[count] = confusion_matrix(error_dt[:,1], predict_dt[:])
    
    
    #####################################################
    ####SVM
    #Treinamento SVM
    pred_svm = autoencoder.predict(knnTrain)
    mse_svm = np.mean(np.power(knnTrain - pred_svm, 2), axis=1)
    error_df_svm = pd.DataFrame({'reconstruction_error': mse_svm,
                                 'true_class': knnTrain_class})
    error_svm = error_df_svm.to_numpy()
    clf = svm.SVC(kernel='rbf')
    clf.fit(error_svm[:,0].reshape(-1,1), error_svm[:,1])

    #Teste
    pred_svm = autoencoder.predict(modelTest)
    mse_svm = np.mean(np.power(modelTest - pred_svm, 2), axis=1)
    error_df_svm = pd.DataFrame({'reconstruction_error': mse_svm,
                                 'true_class': modelTest_class})
    error_svm = error_df_svm.to_numpy()
    predict_svm = clf.predict(error_svm[:,0].reshape(-1,1))
    
    #Métricas
    accuracy_svm[count] = accuracy_score(error_svm[:,1], predict_svm[:])
    recall_svm[count] = recall_score(error_svm[:,1], predict_svm[:])
    precision_svm[count] = precision_score(error_svm[:,1], predict_svm[:])
    f1_svm[count] = f1_score(error_svm[:,1], predict_svm[:])
    matrix_svm[count] = confusion_matrix(error_svm[:,1], predict_svm[:])
    
    count = count + 1
    fold_no = fold_no + 1

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
12894/12894 [==============================] - 26s 2ms/step - loss: 0.0145
Epoch 2/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0090
Epoch 3/50
12894/12894 [==============================] - 16s 1ms/step - loss: 0.0089
Epoch 4/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0088
Epoch 5/50
12894/12894 [==========

12894/12894 [==============================] - 20s 2ms/step - loss: 0.0381
Epoch 35/50
12894/12894 [==============================] - 20s 2ms/step - loss: 0.0381
Epoch 36/50
12894/12894 [==============================] - 22s 2ms/step - loss: 0.0381
Epoch 37/50
12894/12894 [==============================] - 21s 2ms/step - loss: 0.0381
Epoch 38/50
12894/12894 [==============================] - 20s 2ms/step - loss: 0.0381
Epoch 39/50
12894/12894 [==============================] - 19s 1ms/step - loss: 0.0381
Epoch 40/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0381
Epoch 41/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0381
Epoch 42/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0381
Epoch 43/50
12894/12894 [==============================] - 21s 2ms/step - loss: 0.0381
Epoch 44/50
12894/12894 [==============================] - 24s 2ms/step - loss: 0.0381
Epoch 45/50
12894/12894 [==============================

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
12894/12894 [==============================] - 21s 1ms/step - loss: 0.0143
Epoch 2/50
12894/12894 [==============================] - 32s 2ms/step - loss: 0.0085
Epoch 3/50
12894/12894 [==============================] - 30s 2ms/step - loss: 0.0078
Epoch 4/50
12894/12894 [==============================] - 26s 2ms/step - loss: 0.0076
Epoch 5/50
12894/12894 [==========

12894/12894 [==============================] - 21s 2ms/step - loss: 0.0346
Epoch 36/50
12894/12894 [==============================] - 21s 2ms/step - loss: 0.0346
Epoch 37/50
12894/12894 [==============================] - 19s 2ms/step - loss: 0.0346
Epoch 38/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0346
Epoch 39/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0346
Epoch 40/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0346
Epoch 41/50
12894/12894 [==============================] - 19s 1ms/step - loss: 0.0346
Epoch 42/50
12894/12894 [==============================] - 21s 2ms/step - loss: 0.0346
Epoch 43/50
12894/12894 [==============================] - 20s 2ms/step - loss: 0.0346
Epoch 44/50
12894/12894 [==============================] - 20s 2ms/step - loss: 0.0346
Epoch 45/50
12894/12894 [==============================] - 20s 2ms/step - loss: 0.0346
Epoch 46/50
12894/12894 [==============================

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
12894/12894 [==============================] - 24s 2ms/step - loss: 0.0142
Epoch 2/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0110
Epoch 3/50
12894/12894 [==============================] - 19s 1ms/step - loss: 0.0106
Epoch 4/50
12894/12894 [==============================] - 19s 1ms/step - loss: 0.0104
Epoch 5/50
12894/12894 [==========

In [5]:
print("Autoencoder - Limiar Referência:\n",np.mean(matrix_ErroRecons, axis=0), "\nAcurácia: ", np.mean(accuracy_ErroRecons),' +/- ',np.std(accuracy_ErroRecons), "\nRecall: ", np.mean(recall_ErroRecons),' +/- ',np.std(recall_ErroRecons), "\nPrecision: ", np.mean(precision_ErroRecons),' +/- ',np.std(precision_ErroRecons), "\nF1: ", np.mean(f1_ErroRecons),' +/- ',np.std(f1_ErroRecons))
for i in range(0,fold_no-1):
    print("\nFold ",i+1,": " , bestTeta[i], "=", bestmean[i], "+", bestZ[i], "*", bestdesv[i], "\n\nNúmero de Erros: ", bestErrorNumber[i])

Autoencoder - Limiar Referência:
 [[1.590000e+01 3.180000e+01]
 [2.069138e+05 1.598907e+05]] 
Acurácia:  0.4358884404815525  +/-  0.23689939799549997 
Recall:  0.435901717972219  +/-  0.23690939341325365 
Precision:  0.9970580739767849  +/-  0.00560196823809029 
F1:  0.5608930497150479  +/-  0.28374995182524343

Fold  1 :  0.011295359226334243 = 0.011295359226334243 + 0.0 * 1.1328804593813797 

Número de Erros:  364020.0

Fold  2 :  0.04097467291364514 = 0.04097467291364514 + 0.0 * 0.018184176025842717 

Número de Erros:  172200.0

Fold  3 :  0.03818589232939912 = 0.03818589232939912 + 0.0 * 0.06690704222012805 

Número de Erros:  151014.0

Fold  4 :  0.03744744148691997 = 0.03744744148691997 + 0.0 * 0.027456489808059176 

Número de Erros:  178132.0

Fold  5 :  0.0073173234801568255 = 0.0073173234801568255 + 0.0 * 0.021345740958539857 

Número de Erros:  212295.0

Fold  6 :  0.02515337026753736 = 0.02515337026753736 + 0.0 * 0.11487463197030195 

Número de Erros:  74173.0

Fold  7 :  0.

In [6]:
print("Autoencoder - Erro de Reconstrução:\n", np.mean(matrix_ErroRecons, axis=0), "\nAcurácia: ", np.mean(accuracy_ErroRecons),' +/- ',np.std(accuracy_ErroRecons), "\nRecall: ", np.mean(recall_ErroRecons),' +/- ',np.std(recall_ErroRecons), "\nPrecision: ", np.mean(precision_ErroRecons),' +/- ',np.std(precision_ErroRecons), "\nF1: ", np.mean(f1_ErroRecons),' +/- ',np.std(f1_ErroRecons))

print("\nAutoencoder - KNN:\n", np.mean(matrix_knn, axis=0), "\nAcurácia: ", np.mean(accuracy_knn), ' +/- ', np.std(accuracy_knn), "\nRecall: ", np.mean(recall_knn), ' +/- ', np.std(recall_knn), "\nPrecision: ", np.mean(precision_knn), ' +/- ', np.std(precision_knn), "\nF1: ", np.mean(f1_knn), ' +/- ', np.std(f1_knn))

print("\nAutoencoder - SVM:\n", np.mean(matrix_svm, axis=0), "\nAcurácia: ", np.mean(accuracy_svm), ' +/- ', np.std(accuracy_svm), "\nRecall: ", np.mean(recall_svm), ' +/- ', np.std(recall_svm), "\nPrecision: ", np.mean(precision_svm), ' +/- ', np.std(precision_svm), "\nF1: ", np.mean(f1_svm), ' +/- ', np.std(f1_svm))

print("\nAutoencoder - Decision Tree:\n", np.mean(matrix_dt, axis=0), "\nAcurácia: ", np.mean(accuracy_dt), ' +/- ', np.std(accuracy_dt), "\nRecall: ", np.mean(recall_dt), ' +/- ', np.std(recall_dt), "\nPrecision: ", np.mean(precision_dt), ' +/- ', np.std(precision_dt), "\nF1: ", np.mean(f1_dt), ' +/- ', np.std(f1_dt))

Autoencoder - Erro de Reconstrução:
 [[1.590000e+01 3.180000e+01]
 [2.069138e+05 1.598907e+05]] 
Acurácia:  0.4358884404815525  +/-  0.23689939799549997 
Recall:  0.435901717972219  +/-  0.23690939341325365 
Precision:  0.9970580739767849  +/-  0.00560196823809029 
F1:  0.5608930497150479  +/-  0.28374995182524343

Autoencoder - KNN:
 [[4.320000e+01 4.500000e+00]
 [3.920440e+04 3.276001e+05]] 
Acurácia:  0.8931207241672853  +/-  0.07655559298938545 
Recall:  0.893119134232875  +/-  0.07655980978008697 
Precision:  0.9999856016524262  +/-  1.242686656177478e-05 
F1:  0.9417928949622176  +/-  0.04314116798744895

Autoencoder - SVM:
 [[3.600000e+01 1.170000e+01]
 [5.491000e+04 3.118945e+05]] 
Acurácia:  0.8502893341361617  +/-  0.14167603380120156 
Recall:  0.8503018224768489  +/-  0.14169385176644225 
Precision:  0.9999616703704918  +/-  2.7530873145055e-05 
F1:  0.9123887808168595  +/-  0.08757504826799305

Autoencoder - Decision Tree:
 [[4.21000e+01 5.60000e+00]
 [3.90435e+04 3.27761e+

In [9]:
Result_GridSearchLimiar = np.array(["Theta: " , bestTeta,"",
                                "Média: ", bestmean,"",
                                "Valor Z: ", bestZ,"",
                                "Desvio Padrao: ", bestdesv,"",
                                "Número de Erros: ", bestErrorNumber,"",
                                "Erros por Fold: ", ErrorNumberFolds])

np.savetxt('Result_GridSearchLimiarRef_Error.txt', Result_GridSearchLimiar, delimiter = ',',fmt='%s')

C:\Users\luang\anaconda3\envs\keras_env\lib\site-packages\ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [7]:
Result = np.array(["Referência", "Acurácia", accuracy_ErroRecons, np.mean(accuracy_ErroRecons), np.std(accuracy_ErroRecons),"",
                                "Recall", recall_ErroRecons, np.mean(recall_ErroRecons), np.std(recall_ErroRecons), "",
                                "Precision", precision_ErroRecons, np.mean(precision_ErroRecons), np.std(precision_ErroRecons), "",
                                "F1", f1_ErroRecons, np.mean(f1_ErroRecons), np.std(f1_ErroRecons), "-----------------","-----------------",
                   "KNN", "Acurácia", accuracy_knn, np.mean(accuracy_knn), np.std(accuracy_knn),"",
                                "Recall", recall_knn, np.mean(recall_knn), np.std(recall_knn), "",
                                "Precision", precision_knn, np.mean(precision_knn), np.std(precision_knn), "",
                                "F1", f1_knn, np.mean(f1_knn), np.std(f1_knn), "-----------------","-----------------",
                  "Decision Tree",  "Acurácia", accuracy_dt, np.mean(accuracy_dt), np.std(accuracy_dt),"",
                                "Recall", recall_dt, np.mean(recall_dt), np.std(recall_dt), "",
                                "Precision", precision_dt, np.mean(precision_dt), np.std(precision_dt), "",
                                "F1", f1_dt, np.mean(f1_dt), np.std(f1_dt), "-----------------","-----------------",
                  "SVM",  "Acurácia", accuracy_svm, np.mean(accuracy_svm), np.std(accuracy_svm),"",
                                "Recall", recall_svm, np.mean(recall_svm), np.std(recall_svm), "",
                                "Precision", precision_svm, np.mean(precision_svm), np.std(precision_svm), "",
                                "F1", f1_svm, np.mean(f1_svm), np.std(f1_svm), "","",])

np.savetxt('Result_Error_IoT.txt', Result, delimiter = ',',fmt='%s')

C:\Users\luang\anaconda3\envs\keras_env\lib\site-packages\ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()


In [8]:
Result_Matrix = np.array(["Referência", matrix_ErroRecons, np.mean(matrix_ErroRecons, axis=0), np.std(matrix_ErroRecons, axis=0), "-----------------","-----------------",
                          "KNN", matrix_knn, np.mean(matrix_knn, axis=0), np.std(matrix_knn, axis=0), "-----------------","-----------------",
                          "Decision Tree", matrix_dt, np.mean(matrix_dt, axis=0), np.std(matrix_dt, axis=0), "-----------------","-----------------",
                          "SVM", matrix_svm, np.mean(matrix_svm, axis=0), np.std(matrix_svm, axis=0)])

np.savetxt('Result_Error_Matrix_IoT.txt', Result_Matrix, delimiter = ',',fmt='%s')

C:\Users\luang\anaconda3\envs\keras_env\lib\site-packages\ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.
